### This TESS Double Auction Model simulations

In [ ]:
import numpy as np

def generate_daily_solar_profile(sunrise=6, sunset=20, mean_pmax=15, std_pmax=1, seed=None):
    """
    Generate a 24-hour solar irradiance profile using a cosine model for daylight hours.
    
    Parameters:
    - sunrise: Hour of sunrise (e.g., 6)
    - sunset: Hour of sunset (e.g., 18)
    - mean_pmax: Mean of P_max (e.g., 1000 W/m²)
    - std_pmax: Standard deviation of P_max (e.g., 100 W/m²)
    - seed: Random seed for reproducibility
    
    Returns:
    - irradiance: numpy array of length 24, representing hourly solar irradiance
    - p_max: the actual peak irradiance drawn for the day
    """
    if seed is not None:
        np.random.seed(seed)
    
    hours = np.arange(24 + 1)  # 0 to 25 
    irradiance = np.zeros_like(hours, dtype=float)
    
    t_mid = (sunrise + sunset) / 2
    daylight_duration = sunset - sunrise

    # Draw P_max from normal distribution
    p_max = np.random.normal(loc=mean_pmax, scale=std_pmax)

    # Apply cosine model during daylight hours
    for h in range(24 + 1):
        
        if sunrise <= h <= sunset:
            val = p_max * np.cos(np.pi * (h - t_mid) / daylight_duration)
            irradiance[h] = max(0, val)  # clip negative values to zero

        #ensuring that the 25th hour is the same as the 0th hour
        if (h == 25): val[h] == val[0] 

    return irradiance, p_max

# Example usage
# irradiance_profile, p_max = generate_daily_solar_profile(seed=0)
# print(f"P_max for the day: {p_max:.2f}")
# print("Hourly irradiance values:")
# print(irradiance_profile)

def generate_nday_solar(sunrise=6, sunset=20, mean_pmax=14, std_pmax=1, days= 1):
    """Generate nday solar production"""

    irradiance_profile = []
    p_max_list = []
    for day in range(days): 
        #print(f"Day = {day}")
        p_max = -1 
        while p_max <=0:  #ensures all days have a positive p_max
            power_gen_day, p_max = generate_daily_solar_profile(sunrise, sunset, mean_pmax, std_pmax, day)
        #print(f"P_max for the day: {p_max:.2f}")
        #print("Hourly irradiance values:")


        if day == 0: #we want 25 values for day 1 
            irradiance_profile.extend(power_gen_day)
        else: #we want less than 25 values for day 2 
            irradiance_profile.extend(power_gen_day[1:])

        p_max_list.append(p_max)
    
    return irradiance_profile, p_max_list

irradiance_profile, p_max = generate_nday_solar(sunrise=6, sunset=20, mean_pmax=10, std_pmax = 5, days = 7)

print(f"irradiance_profile = {len(irradiance_profile)}")
print(f"p_max = {p_max}")

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import time
import os



# === Initialize output files ===

failures_path = "failures.csv"

if not os.path.exists(failures_path):
    pd.DataFrame(columns=[
        "v_max", "C_max", "p_max", "seed", "message"
    ]).to_csv(failures_path, index=False)

# results_summary.csv
if not os.path.exists("results_summary.csv"):
    pd.DataFrame(columns=[
        "v_max", "C_max", "p_max", "seed",
        "social_welfare", "binding_eq", "binding_ineq"
    ]).to_csv("results_summary.csv", index=False)

# results_hourly.csv
if not os.path.exists("results_hourly.csv"):
    pd.DataFrame(columns=[
        "v_max", "C_max", "p_max", "seed",
        "hour", "p", "q_s", "q_b", "q_u", "C", "q_d", "s_t"
    ]).to_csv("results_hourly.csv", index=False)


# === Simulation grid ===
#v_max_vals = np.arange(105, 4, -10)
v_max_vals = [10,15]
C_max_vals = [5, 10, 15]
p_max_vals = [1, 5, 8, 10, 15]


for v_max_val in v_max_vals:
    for C_max_val in C_max_vals:
        for p_max_val in p_max_vals:
            seed = int(time.time() * 1000) % (2**32 - 1)
            np.random.seed(seed)

            days = 1
            T = 24 * days + 1
            q_max = 10
            c_u = 5
            C_init = int(round(C_max_val / 2))
            C_final = C_init
            sunrise = 6
            sunset =20 
            mean_pmax = p_max_val 
            std_pmax = 0

            s_t, _ = generate_nday_solar(sunrise , sunset , mean_pmax, std_pmax, days)
            s_t = np.array(s_t)

            def unpack(x):
                return x[0:T], x[T:2*T], x[2*T:3*T], x[3*T:4*T], x[4*T:5*T+1]

            def objective(x):
                p, q_s, q_b, q_u, _ = unpack(x)
                q_d = q_max * (1 - p / v_max_val)
                cs = v_max_val * q_d - (v_max_val / (2 * q_max)) * q_d**2 - p * q_d
                return -np.sum(cs + p*q_s + p*q_b + (p - c_u) * q_u)

            def constraint_eq(x):
                p, q_s, q_b, q_u, C = unpack(x)
                cons = [C[0] - C_init, C[T] - C_final, q_s[0] - q_s[-1], q_b[0] - q_b[-1], q_u[0] - q_u[-1]]
                for t in range(T):
                    q_d_t = q_max * (1 - p[t] / v_max_val)
                    cons += [q_s[t] + q_b[t] + q_u[t] - q_d_t, C[t+1] - (C[t] - q_b[t])]
                return np.array(cons)

            def constraint_ineq(x):
                p, q_s, q_b, q_u, _ = unpack(x)
                ineqs = []
                for t in range(T):
                    q_d_t = q_max * (1 - p[t] / v_max_val)
                    ineqs.append(s_t[t] - q_s[t])
                    IR_d_t = v_max_val * q_d_t - (v_max_val / (2 * q_max)) * q_d_t**2 - p[t] * q_d_t
                    ineqs += [IR_d_t, (p[t] - c_u) * q_u[t], p[t] * q_s[t]]
                IR_b = np.sum(p * q_b)
                ineqs.append(IR_b)
                return np.array(ineqs)

            bounds = (
                [(0, None)] * T + [(0, None)] * T +
                [(-C_max_val, C_max_val)] * T +
                [(0, None)] * T +
                [(0, C_max_val)] * (T + 1)
            )

            x0 = np.ones(5 * T + 1)
            x0[:T] = c_u
            x0[T:2*T] = np.minimum(s_t, q_max)
            x0[2*T:3*T] = 0
            x0[3*T:4*T] = np.maximum(q_max - s_t, 0)
            x0[4*T:5*T+1] = C_init


            start_time = time.time()
            
            try:
                res = minimize(
                    fun=objective,
                    x0=x0,
                    method='SLSQP',
                    bounds=bounds,
                    constraints=[
                        {'type': 'eq', 'fun': constraint_eq},
                        {'type': 'ineq', 'fun': constraint_ineq}
                    ],
                    options={'disp': False, 'maxiter': 8000}
                )
                elapsed = time.time() - start_time

                if res.success and elapsed <= 60:
                    p, q_s, q_b, q_u, C = unpack(res.x)
                    q_d = q_max * (1 - p / v_max_val)
                    welfare = -res.fun
                    eq_vals = constraint_eq(res.x)
                    ineq_vals = constraint_ineq(res.x)
                    tol = 1e-6
                    binding_eq = list(np.where(np.abs(eq_vals) > tol)[0])
                    binding_ineq = list(np.where(np.abs(ineq_vals) < tol)[0])

                    summary_data = pd.DataFrame([{
                        "v_max": v_max_val,
                        "C_max": C_max_val,
                        "p_max": p_max_val,
                        "seed": seed,
                        "social_welfare": welfare,
                        "binding_eq": str(binding_eq),
                        "binding_ineq": str(binding_ineq)
                    }])
                    summary_data.to_csv("results_summary.csv", mode='a', header=False, index=False)

                    hourly_data = pd.DataFrame({
                        "v_max": v_max_val,
                        "C_max": C_max_val,
                        "p_max": p_max_val,
                        "seed": seed,
                        "hour": np.arange(T),
                        "p": p,
                        "q_s": q_s,
                        "q_b": q_b,
                        "q_u": q_u,
                        "C": C[:-1],
                        "q_d": q_d,
                        "s_t": s_t
                    })
                    hourly_data.to_csv("results_hourly.csv", mode='a', header=False, index=False)

                elif not res.success or elapsed > 60:
                    fail_message = res.message if not res.success else "timeout"
                    pd.DataFrame([{
                        "v_max": v_max_val,
                        "C_max": C_max_val,
                        "p_max": p_max_val,
                        "seed": seed,
                        "message": fail_message
                    }]).to_csv(failures_path, mode='a', header=False, index=False)

            except Exception as e:
                pd.DataFrame([{
                    "v_max": v_max_val,
                    "C_max": C_max_val,
                    "p_max": p_max_val,
                    "seed": seed,
                    "message": str(e)
                }]).to_csv(failures_path, mode='a', header=False, index=False)


In [ ]:
import pandas as pd
import plotly.express as px

# Load the results summary data
results_summary = pd.read_csv("results_summary.csv")

# Scatter plot: Social Welfare vs v_max
fig_vmax = px.scatter(results_summary, x="v_max", y="social_welfare", color="C_max",
                      symbol="p_max", title="Social Welfare vs v_max (by C_max and p_max)")
fig_vmax.update_layout(xaxis_title="v_max", yaxis_title="Social Welfare")
fig_vmax.show()

# Scatter plot: Social Welfare vs C_max
fig_cmax = px.scatter(results_summary, x="C_max", y="social_welfare", color="v_max",
                      symbol="p_max", title="Social Welfare vs C_max (by v_max and p_max)")
fig_cmax.update_layout(xaxis_title="C_max", yaxis_title="Social Welfare")
fig_cmax.show()

# Scatter plot: Social Welfare vs p_max
fig_pmax = px.scatter(results_summary, x="p_max", y="social_welfare", color="v_max",
                      symbol="C_max", title="Social Welfare vs p_max (by v_max and C_max)")
fig_pmax.update_layout(xaxis_title="p_max", yaxis_title="Social Welfare")
fig_pmax.show()


$v(q) = \int_{0}^{q} v_{max}(1 - \frac{q}{q_{max}})$



In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd


# === Parameters ===
days = 7
T = 24*days + 1
v_max = 10
q_max = 10
c_u = 5
C_init = 0
C_final = 0
C_max = 10

# === Simulated solar irradiance ===
sunrise = 6 
sunset = 20
mean_pmax = 15
std_pmax = 0

solar_gen_profile, p_max_list = generate_nday_solar(sunrise, sunset, mean_pmax, std_pmax, days)

#np.random.seed(0)
s_t = np.array(solar_gen_profile)  
#s_t = np.zeros(T)

# === Unpacking function ===
def unpack(x):
    p = x[0:T]
    q_s = x[T:2*T]
    q_b = x[2*T:3*T]
    q_u = x[3*T:4*T]
    C = x[4*T:5*T+1]
    return p, q_s, q_b, q_u, C

# === Objective function ===
def objective(x):
    p, q_s, q_b, q_u, _ = unpack(x)
    q_d = q_max * (1 - p / v_max)

    # Consumer surplus (area under demand curve minus expenditure)
    cs = v_max * q_d - (v_max / (2 * q_max)) * q_d**2 - p * q_d

    # Payments to solar + battery
    producer_rev = p * q_s
    battery_rev = p*q_b

    # Utility company profit
    utility_profit = (p - c_u) * q_u

    return -np.sum(cs + producer_rev + utility_profit + battery_rev)
    
   

# === Equality Constraints ===
def constraint_eq(x):
    p, q_s, q_b, q_u, C = unpack(x)
    cons = []

    cons.append(C[0] - C_init)
    cons.append(C[T] - C_final) 

    # Final hour values match initial hour values (cyclic constraint)
    cons.append(q_s[0] - q_s[-1])
    cons.append(q_b[0] - q_b[-1])
    cons.append(q_u[0] - q_u[-1])


    for t in range(T):
        q_d_t = q_max * (1 - p[t] / v_max)
        cons.append(q_s[t] + q_b[t] + q_u[t] - q_d_t)
        cons.append(C[t+1] - (C[t] - q_b[t]))
    
    return np.array(cons)

# === Inequality Constraints ===
def constraint_ineq(x):
    p, q_s, q_b, q_u, _ = unpack(x)
    ineqs = []

    for t in range(T):
        q_d_t = q_max * (1 - p[t] / v_max)

        # Physical limits
        ineqs.append(s_t[t] - q_s[t])        # solar capacity
        # ineqs.append(q_s[t])                 # solar ≥ 0
        # ineqs.append(q_u[t])                 # utility ≥ 0
        # ineqs.append(p[t])                   # price ≥ 0

        # IR constraints per period
        IR_d_t = v_max * q_d_t - (v_max / (2 * q_max)) * q_d_t**2 - p[t] * q_d_t
        ineqs.append(IR_d_t)
        ineqs.append((p[t] - c_u) * q_u[t])  # utility IR
        ineqs.append(p[t] * q_s[t])          # solar IR

    # Battery IR across time
    IR_b = np.sum(p * q_b)
    ineqs.append(IR_b)

    return np.array(ineqs)

# === Bounds ===
bounds = (
    [(0, None)] * T +       # Prices ≥ 0
    [(0, None)] * T +       # Solar quantities ≥ 0
    [(-1, 1)] * T + # Battery can charge (-) or discharge (+)
    [(0, None)] * T +       # Utility quantities ≥ 0
    [(0, C_max)] * (T + 1)  # Battery state of charge
)


# === Initial Guess ===
x0 = np.ones(5*T + 1) * c_u
# x0[:T] *= c_u  # prices initially set to marginal cost
# x0[T:2*T] = s_t   # solar dispatch initially matches solar availability
x0[4*T] = C_init  # Battery initial SoC
x0[2*T:3*T] = 0   # Start neutral, let optimizer decide charge/discharge
x0[5*T] = C_final

#===Educated Guess====
# x0 = np.ones(5*T + 1)
# x0[:T] *= c_u# prices initially set to marginal cost
# x0[T:2*T] = s_t   # solar dispatch initially matches solar availability
# x0[2*T:3*T] = 0    # battery initially neutral (no dispatch)
# x0[3*T:4*T] = np.min(abs(5 - s_t), 0) # utility initially covers residual
# x0[4*T:5*T+1] = C_init  # Battery SOC initially constant


# === Solve ===
res = minimize(
    fun=objective,
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=[
        {'type': 'eq', 'fun': constraint_eq},
        {'type': 'ineq', 'fun': constraint_ineq}
    ],
    options={'disp': True, 'maxiter': 8000}
)

# === Output ===
if res.success:
    p, q_s, q_b, q_u, C = unpack(res.x)
    q_d = q_max * (1 - p / v_max)
    df_opt = pd.DataFrame({
        'Hour': np.arange(T),
        'Price': p,
        'q_d': q_d,
        'q_s': q_s,
        'q_b': q_b,
        'q_u': q_u,
        'Battery': C[:-1]
    })
    print(df_opt.head())
    print(f"\nOptimal Welfare: {-res.fun:.3f}")

    # Check which constraints are binding
    eq_vals = constraint_eq(res.x)
    ineq_vals = constraint_ineq(res.x)
    tol = 1e-6  # tolerance for binding

    binding_eq = np.where(np.abs(eq_vals) > tol)[0]
    binding_ineq = np.where(np.abs(ineq_vals) < tol)[0]

    print(f"\nBinding equality constraints indices: {binding_eq}")
    print(f"Binding inequality constraints indices: {binding_ineq}")
else:
    print("Optimization failed:", res.message)


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo

fig = go.Figure()
fig.add_trace(go.Scatter(y=p, mode='lines', name='Price p'))
fig.update_layout(
    title='Price (p) over Time Steps',
    xaxis_title='Time Step',
    yaxis_title='Price (p)'
)
fig.show()


In [ ]:
import plotly.graph_objs as go

fig = go.Figure()

import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(y=q_s, mode='lines', name='Solar Dispatch', line=dict(color='green')))
fig.add_trace(go.Scatter(y=q_b, mode='lines', name='Battery Dispatch', line=dict(color='red')))
fig.add_trace(go.Scatter(y=q_u, mode='lines', name='Utility Dispatch', line=dict(color='purple')))
fig.add_trace(go.Scatter(y=q_d, mode='lines', name='Total Demand', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=C, mode='lines', name='Total Charge (C)'))
fig.add_trace(go.Scatter(y=s_t, mode='lines', name='Solar Supply', line=dict(color='gold', dash='dash')))


fig.update_layout(
    title='Quantity of Energy Dispatched over Time Steps',
    xaxis_title='Time Step',
    yaxis_title='Q (Mwh)'
)
fig.show()


In [ ]:
import numpy as np

def equilibrium_with_battery_offer(
    s_t: float,
    q_b_offer: float,
    v_max: float,
    q_max: float,
    c_u: float,
    q_u_max: float
):
    """
    Compute equilibrium price/quantities given:
      - solar supply at price 0: s_t
      - battery supply at price 0: q_b_offer
      - utility supply at price c_u: capacity q_u_max
      - linear inverse‐demand v(q)=v_max*(1 - q/q_max)

    Returns:
      p_star, q_d, q_s, q_b, q_u
    """
    # 1) total zero‐price supply
    free_supply = s_t + q_b_offer

    # 2) demand at zero price
    q_d0 = q_max  # v(0)=v_max>0 ⇒ q at p=0 is q_max

    # 3) if demand <= free_supply, clear at p=0
    if q_d0 <= free_supply:
        q_d = q_d0
        p_star = 0.0
        # allocate: solar up to its capacity
        q_s = min(s_t, q_d)
        q_b = q_d - q_s
        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 4) check whether the marginal value at free_supply is below c_u
    #    if so, price remains 0 but quantity = free_supply
    v_at_free = v_max * (1 - free_supply / q_max)
    if v_at_free < c_u:
        q_d = free_supply
        p_star = 0.0
        q_s = s_t
        q_b = q_b_offer
        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 5) otherwise bring utility online at cost c_u
    #    demand at p=c_u
    q_d_cu = q_max * (1 - c_u / v_max)
    total_capacity = free_supply + q_u_max

    # 5a) if that demand exceeds total capacity, capacity‐constrained
    if q_d_cu > total_capacity:
        q_d = total_capacity
        p_star = v_max * (1 - q_d / q_max)
    else:
        q_d = q_d_cu
        p_star = c_u

    # 6) allocate across sources in dispatch order
    #    solar first, then battery, then utility
    remaining = q_d
    q_s = min(s_t, remaining)
    remaining -= q_s

    q_b = min(q_b_offer, remaining)
    remaining -= q_b

    q_u = min(q_u_max, remaining)
    remaining -= q_u

    # remaining should be ~0
    return p_star, q_d, q_s, q_b, q_u


def equilibrium_with_battery_bid(
    s_t: float,
    q_b_bid: float,
    v_max: float,
    q_max: float,
    c_u: float,
    q_u_max: float
):
    """
    Supply stack:
      1) Solar at p=0, capacity s_t
      2) Utility at p=c_u, capacity q_u_max

    Demand:
      1) Battery bids q_b_bid at p=0 (charges only if price=0 and solar > consumer)
      2) Consumers with inverse‐demand v(q)=v_max*(1 - q/q_max)

    Returns:
      p_star : equilibrium price
      q_d    : consumer quantity
      q_s    : solar dispatched
      q_b    : battery charged (bid filled)
      q_u    : utility dispatched
    """
    # Consumer demand at zero price
    q_d0 = q_max  # since v(q)=0 => q=q_max
    
    # 1) Check if market clears at p=0 on solar alone:
    if s_t >= q_d0:
        # Solar > consumer demand: price=0, consumers get q_max, battery fills from excess
        p_star = 0.0
        q_d = q_d0
        # battery charges only from leftover solar
        q_b = min(q_b_bid, s_t - q_d)
        q_s = q_d + q_b 

        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 2) Check if solar alone is binding but consumers' marginal value at s_t is below utility cost:
    #    v(s_t) < c_u ⇒ no one pays c_u, so price still 0, consumers get only solar, battery can't charge
    v_at_solar = v_max * (1 - s_t / q_max)
    if v_at_solar < c_u:
        p_star = 0.0
        q_d = s_t
        q_s = s_t
        q_b = 0.0        # no excess to charge into battery
        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 3) Otherwise utility steps in at p=c_u
    #    Find consumers' demand at that price
    q_d_cu = q_max * (1 - c_u / v_max)
    total_capacity = s_t + q_u_max

    # 3a) If demand > total capacity, clear at choke price > c_u
    if q_d_cu > total_capacity:
        q_d = total_capacity
        p_star = v_max * (1 - q_d / q_max)
    else:
        # clear at utility price
        q_d = q_d_cu
        p_star = c_u

    # 4) Allocate dispatch:
    #    solar first (always price=0), then utility (battery can't charge at p>0)
    remaining = q_d
    q_s = min(s_t, remaining)
    remaining -= q_s

    q_b = 0.0     # battery bid only fills at p=0
    q_u = min(q_u_max, remaining)
    remaining -= q_u

    return p_star, q_d, q_s, q_b, q_u


# # Example
# if __name__ == "__main__":
#     s_t        = 50.0   # solar
#     q_b_bid    = 20.0   # battery bid at p=0
#     v_max, q_max = 10.0, 100.0
#     c_u, q_u_max = 5.0,  40.0

#     p_star, q_d, q_s, q_b, q_u = equilibrium_with_battery_bid(
#         s_t, q_b_bid, v_max, q_max, c_u, q_u_max
#     )
#     print(f"p*={p_star:.2f}, consumers={q_d:.2f}, solar={q_s:.2f}, battery_buy={q_b:.2f}, utility={q_u:.2f}")


# Example usage:
if __name__ == "__main__":
    s_t        = 115 # solar
    q_b_offer  = 10.0    # battery offer
    q_b_bid = 10
    v_max, q_max = 10.0, 100.0
    c_u, q_u_max = 5.0,  100

    p_star, q_d, q_s, q_b, q_u = equilibrium_with_battery_offer(
        s_t, q_b_offer, v_max, q_max, c_u, q_u_max
    )

    p_star, q_d, q_s, q_b, q_u = equilibrium_with_battery_bid(
        s_t, q_b_bid, v_max, q_max, c_u, q_u_max
    )
    print(f"p*={p_star:.2f}, q_d={q_d:.2f}, q_s={q_s:.2f}, q_b={q_b:.2f}, q_u={q_u:.2f}")
    v = v_max*(1- (q_b + q_s)/q_max)
    print(v)


In [ ]:
from itertools import product
import numpy as np
import pandas as pd

In [ ]:
sim_df.iloc[0]

In [ ]:
days = 7
T = 24 * days 
v_max = 10
q_max = 10
c_u = 5
q_u_max = q_max
C_init = 1
C_final = 1
C_max = 5

C_max_grid = np.arange(5, 21)
mean_pmax_grid = np.arange(5, 21)
std_pmax_grid = np.arange(1, 6)
# Predefine sunrise/sunset if needed
sunrise = 6
sunset = 20


C_grid = np.linspace(0, C_max, C_max +1)      # SOC: 0, 1, ..., 10
q_b_grid = np.linspace(-10, 10, 21)    # Battery dispatch: -5, -4, ..., 5
q_b_grid = np.linspace(-3, 3, 2*3 +1 ) 
q_b_grid = np.arange(-1, 2, 1) 

# Error tolerance: half your grid spacing
#atol = (C_grid[1] - C_grid[0]) / 2   # e.g., 0.25 if grid step is 0.5
atol = 0
beta = 1 
mean_pmax = 5
std_pmax = 0

solar_gen_profile, _ = generate_nday_solar(
    sunrise, sunset, mean_pmax, std_pmax=std_pmax, days=days
)
s_t = np.array(solar_gen_profile)

V = np.zeros((len(C_grid), T+1))
P = np.zeros_like(V)
QS = np.zeros_like(V)
QB = np.zeros_like(V)
QU = np.zeros_like(V)

# Terminal constraint (battery returns to initial state)
V[:, T] = -np.inf
i_init = np.searchsorted(C_grid, C_init)
V[i_init, T] = 0


for t in reversed(range(T)):
    solar = s_t[t]
    for i, C in enumerate(C_grid):
        best = -np.inf
        bp = bqs = bqb = bqu = 0.0

        for q_b_val in q_b_grid:
            C_next = C - q_b_val
            
            # Ensure C_next is exactly at C_init when at final step (t = T-1)
            if t == T - 1 and not np.isclose(C_next, C_init, atol=atol):
                continue
            
            # ensure state is within bounds for all other time steps
            if not (0 <= C_next <= C_max):
                continue

            if q_b_val < 0:
                p_star, q_d, q_s, q_b_ex, q_u = equilibrium_with_battery_bid(
                    solar, -q_b_val, v_max, q_max, c_u, q_u_max
                )
                q_b = -q_b_ex
                
            else:
                p_star, q_d, q_s, q_b_ex, q_u = equilibrium_with_battery_offer(
                    solar, q_b_val, v_max, q_max, c_u, q_u_max
                )
                q_b = q_b_ex

            inst = p_star * q_b_ex

            idx = np.searchsorted(C_grid, C_next)
            idx = min(max(idx, 0), len(C_grid)-1)

            val = inst + beta * V[idx, t+1]

            if val > best:
                best = val
                bp, bqs, bqb, bqu = p_star, q_s, q_b, q_u

        V[i, t] = best
        P[i, t], QS[i, t], QB[i, t], QU[i, t] = bp, bqs, bqb, bqu





In [ ]:
import numpy as np
import pandas as pd
from itertools import product

In [ ]:


# Parameter ranges
C_max_grid = [5,10,15,20]        # C_max: 5..20
mean_pmax_grid = [5,10,15,20]# mean_pmax: 5..20
std_pmax_grid = [0]    # std_pmax: 1..5
days = 7
T = 24 * days
sunrise = 6
sunset = 20

results = []
period_rows = []

for C_max in C_max_grid:
    # For each C_max, C_init ranges from 0 up to C_max (inclusive)
    C_init_grid = np.arange(0, C_max + 1)
    # For each C_max, q_b_max ranges from 1 up to C_max (inclusive)
    q_b_max_grid = np.arange(1, C_max + 1)

    for C_init, q_b_max, mean_pmax, std_pmax in product(
        C_init_grid, q_b_max_grid, mean_pmax_grid, std_pmax_grid
    ):
        # Setup grids
        C_grid = np.arange(0, C_max + 1) #integer 
        q_b_grid = np.arange(-q_b_max, q_b_max + 1) #integer 

        # Solar generation
        solar_gen_profile, _ = generate_nday_solar(
            sunrise, sunset, mean_pmax, std_pmax, days=days
        )
        s_t = np.array(solar_gen_profile[:T])

        # Initialize value function arrays
        V = np.zeros((len(C_grid), T+1))
        P = np.zeros_like(V)
        QS = np.zeros_like(V)
        QB = np.zeros_like(V)
        QU = np.zeros_like(V)

        # Terminal constraint: return to initial SOC
        V[:, T] = -np.inf
        i_init = np.searchsorted(C_grid, C_init)
        V[i_init, T] = 0

        # Value function iteration loop (same as before)
        for t in reversed(range(T)):
            solar = s_t[t]
            for i, C in enumerate(C_grid):
                best = -np.inf
                bp = bqs = bqb = bqu = 0.0
                for q_b_val in q_b_grid:
                    C_next = C - q_b_val
                    if t == T - 1 and not np.isclose(C_next, C_init, atol=0):
                        continue
                    if not (0 <= C_next <= C_max):
                        continue

                    # Insert your equilibrium functions here!
                    p_star, q_d, q_s, q_b_ex, q_u = 0, 0, 0, 0, 0  # TODO

                    inst = p_star * q_b_ex
                    idx = np.searchsorted(C_grid, C_next)
                    idx = min(max(idx, 0), len(C_grid)-1)
                    val = inst + V[idx, t+1]
                    if val > best:
                        best = val
                        bp, bqs, bqb, bqu = p_star, q_s, q_b_ex, q_u
                V[i, t] = best
                P[i, t], QS[i, t], QB[i, t], QU[i, t] = bp, bqs, bqb, bqu

        # Forward pass for optimal policy
        i = i_init
        C = C_init
        prices = []
        socs = []
        q_s_list = []
        q_b_list = []
        q_u_list = []
        q_d_list = []
        surplus_battery = []
        surplus_utility = []
        surplus_solar = []
        surplus_demand = []
        surplus_total = []

        for t in range(T):
            socs.append(C)
            prices.append(P[i, t])
            q_s_list.append(QS[i, t])
            q_b_list.append(QB[i, t])
            q_u_list.append(QU[i, t])

            #Determining Demand 
            p = P[i,t] #current price
            q_d = q_max*(1-p/v_max)
            q_d_list.append(q_d)

            # Surplus calculations 
            surplus_battery.append(P[i, t] * QB[i, t])  
            surplus_solar.append(P[i, t] * QS[i, t])   
            surplus_utility.append((P[i, t] - c_u) * QS[i, t])  
            surplus_demand.append(v_max * q_d - (v_max / (2 * q_max)) * q_d**2 - p * q_d)                  
            surplus_total.append(surplus_battery[-1] + surplus_utility[-1] + surplus_demand[-1] + surplus_solar[-1])

            #Advance to next state 
            C = C - QB[i, t]
            i = np.searchsorted(C_grid, C)
            i = min(max(i, 0), len(C_grid)-1)

        #Ensure a closed loop, the first period T=0 matches the final period T+1
        socs.append(C)
        prices.append(prices[0])
        q_s_list.append(q_s_list[0])
        q_b_list.append(q_b_list[0])
        q_u_list.append(q_u_list[0])

        q_d_list.append(q_d_list[0])
        # Surplus calculations 
        surplus_battery.append(surplus_battery[0])  
        surplus_solar.append(surplus_solar[0])   
        surplus_utility.append(surplus_utility[0])  
        surplus_demand.append(surplus_demand[0])                  
        surplus_total.append(surplus_battery[0] + surplus_utility[0] + surplus_demand[0] + surplus_solar[0])

        for t in range(T + 1): #Extra period for the closed loop 
            period_rows.append({
                'C_max': C_max,
                'C_init': C_init,
                'q_b_max': q_b_max,
                'mean_pmax': mean_pmax,
                'std_pmax': std_pmax,
                'period': t,
                'price': prices[t],
                'soc': socs[t],
                'q_s': q_s_list[t],
                'q_b': q_b_list[t],
                'q_u': q_u_list[t],
                'surplus_battery': surplus_battery[t],
                'surplus_utility': surplus_utility[t],
                'surplus_demand': surplus_demand[t],
                'surplus_total': surplus_total[t]
            })


        results.append({
            'C_max': C_max,
            'C_init': C_init,
            'q_b_max': q_b_max,
            'mean_pmax': mean_pmax,
            'std_pmax': std_pmax,
            'prices': prices,
            'socs': socs,
            'q_s': q_s_list,
            'q_b': q_b_list,
            'q_u': q_u_list,
            'surplus_battery_ts': surplus_battery,
            'surplus_utility_ts': surplus_utility,
            'surplus_demand_ts': surplus_demand,
            'surplus_total_ts': surplus_total,
            'total_surplus_battery': sum(surplus_battery),
            'total_surplus_utility': sum(surplus_utility),
            'total_surplus_demand': sum(surplus_demand),
            'total_surplus_all': sum(surplus_total)
        })


# At the end, after all runs:
df_summary = pd.DataFrame(results)
df_period = pd.DataFrame(period_rows)



In [ ]:
# Parameter ranges
C_max_grid = [10,15,20]        # C_max: 5..20
mean_pmax_grid = [5,10,15,20]# mean_pmax: 5..20
prod = len(C_max_grid)*len(mean_pmax_grid)

for C_max in C_max_grid:
    # For each C_max, C_init ranges from 0 up to C_max (inclusive)
    C_init_grid = np.arange(0, C_max + 1,5)
    print(C_init_grid)
    # For each C_max, q_b_max ranges from 1 up to C_max (inclusive)
    q_b_max_grid = [1, *np.arange(0, C_max + 1,5)[1:]]
    print(q_b_max_grid)

    prod = prod*len(C_init_grid)*len(q_b_max_grid)
    print(prod)



In [ ]:
import numpy as np
import pandas as pd
from itertools import product
import os
from concurrent.futures import ProcessPoolExecutor, as_completed


# Output file paths
summary_csv = "simulation_summary.csv"
period_csv = " .csv"

# Remove old files if present (optional)
for fname in [summary_csv, period_csv]:
    if os.path.exists(fname):
        os.remove(fname)

In [ ]:


# ---- 1. Simulation function for one parameter combination ----
def run_simulation(args):
    (C_max, C_init, q_b_max, mean_pmax, std_pmax, days, T, sunrise, sunset) = args
    
    period_rows = []
    summary_rows = []

    # Setup grids
    C_grid = np.arange(0, C_max + 1) #integer 
    q_b_grid = np.arange(-q_b_max, q_b_max + 1) #integer 

    # Solar generation
    solar_gen_profile, _ = generate_nday_solar(
        sunrise, sunset, mean_pmax, std_pmax, days=days
    )
    s_t = np.array(solar_gen_profile[:T])

    # Initialize value function arrays
    V = np.zeros((len(C_grid), T+1))
    P = np.zeros_like(V)
    QS = np.zeros_like(V)
    QB = np.zeros_like(V)
    QU = np.zeros_like(V)

    # Terminal constraint: return to initial SOC
    V[:, T] = -np.inf
    i_init = np.searchsorted(C_grid, C_init)
    V[i_init, T] = 0

    # Value function iteration loop (same as before)
    for t in reversed(range(T)):
        solar = s_t[t]
        for i, C in enumerate(C_grid):
            best = -np.inf
            bp = bqs = bqb = bqu = 0.0
            for q_b_val in q_b_grid:
                C_next = C - q_b_val
                if t == T - 1 and not np.isclose(C_next, C_init, atol=0):
                    continue
                if not (0 <= C_next <= C_max):
                    continue

                # Insert your equilibrium functions here!
                p_star, q_d, q_s, q_b_ex, q_u = 0, 0, 0, 0, 0  # TODO

                inst = p_star * q_b_ex
                idx = np.searchsorted(C_grid, C_next)
                idx = min(max(idx, 0), len(C_grid)-1)
                val = inst + V[idx, t+1]
                if val > best:
                    best = val
                    bp, bqs, bqb, bqu = p_star, q_s, q_b_ex, q_u
            V[i, t] = best
            P[i, t], QS[i, t], QB[i, t], QU[i, t] = bp, bqs, bqb, bqu

    # Forward pass for optimal policy
    i = i_init
    C = C_init
    prices = []
    socs = []
    q_s_list = []
    q_b_list = []
    q_u_list = []
    q_d_list = []
    surplus_battery = []
    surplus_utility = []
    surplus_solar = []
    surplus_demand = []
    surplus_total = []

    for t in range(T):
        socs.append(C)
        prices.append(P[i, t])
        q_s_list.append(QS[i, t])
        q_b_list.append(QB[i, t])
        q_u_list.append(QU[i, t])

        #Determining Demand 
        p = P[i,t] #current price
        q_d = q_max*(1-p/v_max)
        q_d_list.append(q_d)

        # Surplus calculations 
        surplus_battery.append(P[i, t] * QB[i, t])  
        surplus_solar.append(P[i, t] * QS[i, t])   
        surplus_utility.append((P[i, t] - c_u) * QS[i, t])  
        surplus_demand.append(v_max * q_d - (v_max / (2 * q_max)) * q_d**2 - p * q_d)                  
        surplus_total.append(surplus_battery[-1] + surplus_utility[-1] + surplus_demand[-1] + surplus_solar[-1])

        #Advance to next state 
        C = C - QB[i, t]
        i = np.searchsorted(C_grid, C)
        i = min(max(i, 0), len(C_grid)-1)

    #Ensure a closed loop, the first period T=0 matches the final period T+1
    socs.append(C)
    prices.append(prices[0])
    q_s_list.append(q_s_list[0])
    q_b_list.append(q_b_list[0])
    q_u_list.append(q_u_list[0])

    q_d_list.append(q_d_list[0])
    # Surplus calculations 
    surplus_battery.append(surplus_battery[0])  
    surplus_solar.append(surplus_solar[0])   
    surplus_utility.append(surplus_utility[0])  
    surplus_demand.append(surplus_demand[0])                  
    surplus_total.append(surplus_battery[0] + surplus_utility[0] + surplus_demand[0] + surplus_solar[0])


    summary_row = {
        'C_max': C_max,
        'C_init': C_init,
        'q_b_max': q_b_max,
        'mean_pmax': mean_pmax,
        'std_pmax': std_pmax,
        'prices': prices,
        'socs': socs,
        'q_s': q_s_list,
        'q_b': q_b_list,
        'q_u': q_u_list,
        'q_d': q_d_list,
        'surplus_battery_ts': surplus_battery,
        'surplus_utility_ts': surplus_utility,
        'surplus_demand_ts': surplus_demand,
        'surplus_total_ts': surplus_total,
        'total_surplus_battery': sum(surplus_battery),
        'total_surplus_utility': sum(surplus_utility),
        'total_surplus_demand': sum(surplus_demand),
        'total_surplus_all': sum(surplus_total)
    }

    for t in range(T + 1): # T+1st period loops around and is the same as first period
        period_rows.append({
            'C_max': C_max,
            'C_init': C_init,
            'q_b_max': q_b_max,
            'mean_pmax': mean_pmax,
            'std_pmax': std_pmax,
            'period': t,
            'price': prices[t],
            'soc': socs[t],
            'q_s': q_s_list[t],
            'q_b': q_b_list[t],
            'q_u': q_u_list[t],
            'q_d': q_d_list[t],
            'surplus_battery': surplus_battery[t],
            'surplus_utility': surplus_utility[t],
            'surplus_demand': surplus_demand[t],
            'surplus_total': surplus_total[t]
        })

    return summary_row, period_rows





# ---- 3. Parallel execution and CSV writing ----
def write_csv_row(df, file, header):
    df.to_csv(file, mode='a', index=False, header=header)




In [ ]:

C_max_grid = [5, 10, 15, 20]
C_init_fracs = [0.2, 0.5, 0.8]             # 20%, 50%, 80%
q_b_max_grid = [1, 2, 3, 4, 5]
mean_pmax_grid = [5, 10, 15, 20]
days = 7
T = 24 * days
sunrise = 6
sunset = 20

param_grid = []
for C_max in C_max_grid:
    # For each C_max, compute all valid integer initial charges
    C_init_grid = [int(round(C_max * frac)) for frac in C_init_fracs]
    for C_init, q_b_max, mean_pmax in product(C_init_grid, q_b_max_grid, mean_pmax_grid):
        std_pmax_grid = [0, mean_pmax / 2]    # For each P_max, try 0 and P_max/2
        for std_pmax in std_pmax_grid:
            param_grid.append(
                (C_max, C_init, q_b_max, mean_pmax, std_pmax, days, T, sunrise, sunset)
            )
print(f"Total simulations: {len(param_grid)}")
# You can inspect the first few rows if desired:
# for p in param_grid[:5]: print(p) 

In [ ]:
from tqdm import tqdm
import multiprocessing

with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
    futures = [executor.submit(run_simulation, args) for args in param_grid]

    # Write headers
    first_summary = True
    first_period = True

    for future in tqdm(as_completed(futures), total=len(futures)):
        summary_row, period_rows = future.result()

        # Write summary row
        summary_df = pd.DataFrame([summary_row])
        write_csv_row(summary_df, summary_csv, header=first_summary)
        first_summary = False

        # Write period rows
        period_df = pd.DataFrame(period_rows)
        write_csv_row(period_df, period_csv, header=first_period)
        first_period = False

print("All simulations complete. Results written to CSV.")

### Potential Variations 


Max Charge $C_{max}$ : [5,10,15,20]   
Initial Charge : [ 20%, 50%, 80%]. (maybe 0 and max because not good for batteries)
Abs max charge/discharge : [1,2,3,4,5]      
Max Solar (Noon) $P_{max}$ : [5,10,15,20]    
Std of Max Solar: [0, $\frac{P_{max}}{2}$]  (use cumulative normal and clip values at 0, beta distribution, linear distribution) 



In [ ]:
sim_df = pd.read_csv("/Users/nalin/Desktop/UChicago/Thesis/simulation_summary.csv")


In [ ]:
sim_df.head()

In [ ]:
import plotly.graph_objects as go
import ast

def plot_summary_row(row, title_suffix=""):
    # Helper to parse the stringified lists
    def parse(col):
        if isinstance(col, str):
            return ast.literal_eval(col)
        return col  # already a list

    prices = parse(row['prices'])
    socs = parse(row['socs'])
    q_s_list = parse(row['q_s'])
    q_b_list = parse(row['q_b'])
    q_u_list = parse(row['q_u'])
    q_d_list = parse(row['q_d'])
    # For solar supply curve, you can use q_s or supply your own s_t if you saved it.

    time_index = list(range(len(prices)))

    # 1. Battery Dispatch (q_b)
    fig_battery = go.Figure()
    fig_battery.add_trace(go.Scatter(x=time_index, y=q_b_list, mode='lines+markers', name='Battery Dispatch', line=dict(color='red')))
    fig_battery.add_hline(y=0, line_dash="dash", line_color="gray")
    fig_battery.update_layout(title=f"Optimal Battery Dispatch{title_suffix}", xaxis_title="Time", yaxis_title="Battery Dispatch (q_b)", 
                            xaxis=dict(range=[0, len(time_index)]))

    # 2. Battery State of Charge (SOC)
    fig_soc = go.Figure()
    fig_soc.add_trace(go.Scatter(x=time_index, y=socs, mode='lines+markers', name='SOC', line=dict(color='green')))
    fig_soc.add_hline(y=socs[0], line_dash="dot", line_color="orange", annotation_text="Initial/Final SOC", annotation_position="bottom right")
    fig_soc.update_layout(title=f"Battery State-of-Charge (SOC){title_suffix}", xaxis_title="Time", yaxis_title="SOC",
                          yaxis=dict(range=[0, max(prices)*1.1]))

    # 3. Market Clearing Price
    fig_price = go.Figure()
    fig_price.add_trace(go.Scatter(x=time_index, y=prices, mode='lines+markers', name='Market Price', line=dict(color='blue')))
    fig_price.update_layout(title=f"Market Clearing Price{title_suffix}", xaxis_title="Time", yaxis_title="Price")

    # 4. Dispatch Quantities
    fig_dispatch = go.Figure()
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_s_list, mode='lines', name='Solar Dispatch', line=dict(color='green')))
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_b_list, mode='lines', name='Battery Dispatch', line=dict(color='red')))
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_u_list, mode='lines', name='Utility Dispatch', line=dict(color='purple')))
    fig_dispatch.add_trace(go.Scatter(x=time_index, y=q_d_list, mode='lines', name='Total Demand', line=dict(color='blue')))
    # Optionally, add Solar Supply if you saved s_t
    if 's_t' in row:
        s_t = parse(row['s_t'])
        fig_dispatch.add_trace(go.Scatter(x=time_index, y=s_t, mode='lines', name='Solar Supply', line=dict(color='orange', dash='dash')))
    
    fig_dispatch.update_layout(
        title=f"Dispatch and Market Quantities{title_suffix}",
        xaxis_title="Time", yaxis_title="Quantity"
    )

    return {
        "battery_dispatch": fig_battery,
        "state_of_charge": fig_soc,
        "market_price": fig_price,
        "dispatch_quantities": fig_dispatch
    }

# Usage example:
# row = summary_df.iloc[0]
# figs = plot_summary_row(row)
# figs['battery_dispatch'].show()
# figs['state_of_charge'].show()
# figs['market_price'].show()
# figs['dispatch_quantities'].show()


In [ ]:
plot_dict = plot_summary_row(sim_df.iloc[452])

In [ ]:
plot_dict.keys()

In [ ]:
plot_dict["state_of_charge"].show()

In [ ]:
plot_dict["market_price"].show()

In [ ]:
plot_dict["dispatch_quantities"].show()

In [ ]:
plot_dict["battery_dispatch"].show()

In [ ]:
row = sim_df[
    (sim_df['C_max'] == 20) &
    (sim_df['C_init'] == 16) &
    (sim_df['q_b_max'] == 2) &
    (sim_df['mean_pmax'] == 20) &
    (sim_df['std_pmax'] == 0)
]

In [ ]:
row
